## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Configure visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print('All libraries imported successfully!')

## 2. Generate Sample Dataset

Since this is a portfolio project, we'll generate a realistic e-commerce dataset for demonstration.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic e-commerce data
n_records = 5000

# Date range: 2019-2024
date_start = pd.to_datetime('2019-01-01')
date_end = pd.to_datetime('2024-12-31')
dates = pd.date_range(start=date_start, end=date_end, periods=n_records)

# Create dataset
data = {
    'order_id': range(1000, 1000 + n_records),
    'customer_id': np.random.randint(1000, 2000, n_records),
    'order_date': dates,
    'category': np.random.choice(['Electronics', 'Home & Garden', 'Fashion', 'Books', 'Sports'], n_records, p=[0.35, 0.28, 0.20, 0.12, 0.05]),
    'quantity': np.random.randint(1, 10, n_records),
    'region': np.random.choice(['North', 'South', 'East', 'West'], n_records),
    'customer_segment': np.random.choice(['Regular', 'Premium', 'New'], n_records, p=[0.60, 0.25, 0.15])
}

# Add sales with some correlation to quantity and category
category_prices = {'Electronics': 300, 'Home & Garden': 150, 'Fashion': 80, 'Books': 25, 'Sports': 120}
data['sales'] = data['quantity'] * data['category'].map(category_prices) + np.random.normal(0, 50, n_records)
data['sales'] = data['sales'].clip(lower=0)  # Ensure non-negative

df = pd.DataFrame(data)

print(f'Dataset generated with {len(df)} records')
print(f'Date range: {df["order_date"].min().date()} to {df["order_date"].max().date()}')
print(f'\nFirst few rows:')
df.head()

## 3. Load and Explore Dataset

In [ ]:
# Display basic information
print('=' * 60)
print('DATASET OVERVIEW')
print('=' * 60)

print(f'\nDataset Shape: {df.shape}')
print(f'\nColumn Names and Types:')
print(df.dtypes)
print(f'\nBasic Statistics:')
df.describe()

In [ ]:
# Check for missing values
print('\n' + '=' * 60)
print('MISSING VALUES ANALYSIS')
print('=' * 60)

missing_data = df.isnull().sum()
missing_percent = (missing_data / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Percentage': missing_percent
})

print('\n', missing_df[missing_df['Missing_Count'] > 0])
print('No missing values found!' if missing_df['Missing_Count'].sum() == 0 else '')

## 4. Data Cleaning and Preprocessing

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Ensure correct data types
df_clean['order_date'] = pd.to_datetime(df_clean['order_date'])

# Remove duplicates
initial_rows = len(df_clean)
df_clean = df_clean.drop_duplicates()
duplicates_removed = initial_rows - len(df_clean)
print(f'Duplicates removed: {duplicates_removed}')

# Handle outliers using IQR method
Q1 = df_clean['sales'].quantile(0.25)
Q3 = df_clean['sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

outliers = df_clean[(df_clean['sales'] < lower_bound) | (df_clean['sales'] > upper_bound)]
print(f'Outliers detected: {len(outliers)}')
df_clean = df_clean[(df_clean['sales'] >= lower_bound) & (df_clean['sales'] <= upper_bound)]

# Extract time features
df_clean['year'] = df_clean['order_date'].dt.year
df_clean['month'] = df_clean['order_date'].dt.month
df_clean['quarter'] = df_clean['order_date'].dt.quarter
df_clean['day_of_week'] = df_clean['order_date'].dt.day_name()
df_clean['is_weekend'] = df_clean['order_date'].dt.dayofweek.isin([5, 6]).astype(int)

# Create derived features
df_clean['price_per_unit'] = df_clean['sales'] / df_clean['quantity']

print(f'\nData cleaning completed!')
print(f'Final dataset shape: {df_clean.shape}')
print(f'\nNew features created: year, month, quarter, day_of_week, is_weekend, price_per_unit')

df_clean.head()

## 5. Exploratory Data Analysis (EDA)

In [ ]:
# Statistical summary
print('=' * 60)
print('DESCRIPTIVE STATISTICS')
print('=' * 60)
print('\nSales Statistics:')
print(f'  Total Revenue: ${df_clean["sales"].sum():,.2f}')
print(f'  Average Order Value: ${df_clean["sales"].mean():,.2f}')
print(f'  Median Order Value: ${df_clean["sales"].median():,.2f}')
print(f'  Standard Deviation: ${df_clean["sales"].std():,.2f}')
print(f'  Min Sale: ${df_clean["sales"].min():,.2f}')
print(f'  Max Sale: ${df_clean["sales"].max():,.2f}')

print('\nCustomer Statistics:')
print(f'  Total Unique Customers: {df_clean["customer_id"].nunique()}')
print(f'  Total Orders: {len(df_clean)}')
print(f'  Average Orders per Customer: {len(df_clean) / df_clean["customer_id"].nunique():.2f}')

In [ ]:
# Category analysis
print('\n' + '=' * 60)
print('SALES BY CATEGORY')
print('=' * 60)

category_analysis = df_clean.groupby('category').agg({
    'sales': ['sum', 'mean', 'count']
}).round(2)

category_analysis.columns = ['Total Sales', 'Avg Sale', 'Count']
category_analysis['Percentage'] = (category_analysis['Total Sales'] / category_analysis['Total Sales'].sum() * 100).round(2)
category_analysis = category_analysis.sort_values('Total Sales', ascending=False)

print('\n', category_analysis)

In [ ]:
# Regional analysis
print('\n' + '=' * 60)
print('SALES BY REGION')
print('=' * 60)

region_analysis = df_clean.groupby('region').agg({
    'sales': ['sum', 'mean', 'count']
}).round(2)

region_analysis.columns = ['Total Sales', 'Avg Sale', 'Count']
region_analysis['Percentage'] = (region_analysis['Total Sales'] / region_analysis['Total Sales'].sum() * 100).round(2)
region_analysis = region_analysis.sort_values('Total Sales', ascending=False)

print('\n', region_analysis)

In [ ]:
# Customer segment analysis
print('\n' + '=' * 60)
print('SALES BY CUSTOMER SEGMENT')
print('=' * 60)

segment_analysis = df_clean.groupby('customer_segment').agg({
    'sales': ['sum', 'mean', 'count'],
    'customer_id': 'nunique'
}).round(2)

segment_analysis.columns = ['Total Sales', 'Avg Sale', 'Orders', 'Customers']
segment_analysis['Avg Orders per Customer'] = (segment_analysis['Orders'] / segment_analysis['Customers']).round(2)
segment_analysis = segment_analysis.sort_values('Total Sales', ascending=False)

print('\n', segment_analysis)

## 6. Data Visualization

In [ ]:
# Distribution of sales
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histogram of sales
axes[0, 0].hist(df_clean['sales'], bins=50, edgecolor='black', color='steelblue', alpha=0.7)
axes[0, 0].set_title('Distribution of Sales Amount', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Sales ($)')
axes[0, 0].set_ylabel('Frequency')

# Box plot of sales
axes[0, 1].boxplot(df_clean['sales'])
axes[0, 1].set_title('Sales Amount - Box Plot', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Sales ($)')

# Distribution of quantity
axes[1, 0].hist(df_clean['quantity'], bins=10, edgecolor='black', color='lightcoral', alpha=0.7)
axes[1, 0].set_title('Distribution of Quantity', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Quantity')
axes[1, 0].set_ylabel('Frequency')

# Box plot of quantity
axes[1, 1].boxplot(df_clean['quantity'])
axes[1, 1].set_title('Quantity - Box Plot', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Quantity')

plt.tight_layout()
plt.show()

print('Sales Distribution Statistics:')
print(f'  Skewness: {stats.skew(df_clean["sales"]):.3f}')
print(f'  Kurtosis: {stats.kurtosis(df_clean["sales"]):.3f}')

In [ ]:
# Sales by category and region
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar chart - Sales by Category
category_sales = df_clean.groupby('category')['sales'].sum().sort_values(ascending=False)
axes[0].bar(category_sales.index, category_sales.values, color='steelblue', alpha=0.8, edgecolor='black')
axes[0].set_title('Total Sales by Category', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Category')
axes[0].set_ylabel('Sales ($)')
axes[0].tick_params(axis='x', rotation=45)
for i, v in enumerate(category_sales.values):
    axes[0].text(i, v + 5000, f'${v:,.0f}', ha='center', va='bottom')

# Bar chart - Sales by Region
region_sales = df_clean.groupby('region')['sales'].sum().sort_values(ascending=False)
axes[1].bar(region_sales.index, region_sales.values, color='lightcoral', alpha=0.8, edgecolor='black')
axes[1].set_title('Total Sales by Region', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Region')
axes[1].set_ylabel('Sales ($)')
for i, v in enumerate(region_sales.values):
    axes[1].text(i, v + 5000, f'${v:,.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Monthly sales trend
monthly_sales = df_clean.groupby(df_clean['order_date'].dt.to_period('M'))['sales'].sum()

fig, ax = plt.subplots(figsize=(14, 6))
monthly_sales.plot(kind='line', marker='o', color='steelblue', linewidth=2, markersize=6, ax=ax)
ax.set_title('Monthly Sales Trend (2019-2024)', fontsize=12, fontweight='bold')
ax.set_xlabel('Month')
ax.set_ylabel('Sales ($)')
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f'\nMonthly Sales Summary:')
print(f'  Average Monthly Sales: ${monthly_sales.mean():,.2f}')
print(f'  Max Monthly Sales: ${monthly_sales.max():,.2f}')
print(f'  Min Monthly Sales: ${monthly_sales.min():,.2f}')

In [ ]:
# Category and Region interaction
fig, ax = plt.subplots(figsize=(12, 6))

category_region = df_clean.groupby(['category', 'region'])['sales'].sum().unstack()
category_region.plot(kind='bar', ax=ax, width=0.8, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax.set_title('Sales by Category and Region', fontsize=12, fontweight='bold')
ax.set_xlabel('Category')
ax.set_ylabel('Sales ($)')
ax.legend(title='Region')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Correlation analysis
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
correlation_matrix = df_clean[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, fmt='.2f', ax=ax, cbar_kws={'label': 'Correlation'})
ax.set_title('Correlation Matrix of Numeric Variables', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print('\nKey Correlations with Sales:')
correlations_with_sales = correlation_matrix['sales'].sort_values(ascending=False)
print(correlations_with_sales)

## 7. Statistical Analysis

In [ ]:
# RFM (Recency, Frequency, Monetary) Analysis
print('=' * 60)
print('RFM ANALYSIS')
print('=' * 60)

# Calculate RFM metrics
current_date = df_clean['order_date'].max()

rfm = df_clean.groupby('customer_id').agg({
    'order_date': lambda x: (current_date - x.max()).days,  # Recency
    'customer_id': 'count',  # Frequency
    'sales': 'sum'  # Monetary
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm[rfm['Monetary'] > 0]

print(f'\nRFM Statistics:')
print(rfm.describe().round(2))

# Segment customers
rfm['R_Quartile'] = pd.qcut(rfm['Recency'], 4, labels=['1', '2', '3', '4'], duplicates='drop')
rfm['F_Quartile'] = pd.qcut(rfm['Frequency'].rank(method='first'), 4, labels=['4', '3', '2', '1'], duplicates='drop')
rfm['M_Quartile'] = pd.qcut(rfm['Monetary'], 4, labels=['4', '3', '2', '1'], duplicates='drop')

rfm['RFM_Segment'] = rfm['R_Quartile'].astype(str) + rfm['F_Quartile'].astype(str) + rfm['M_Quartile'].astype(str)

print(f'\nCustomer Segments: {rfm["RFM_Segment"].nunique()} unique segments')
print(f'\nTop 10 RFM Segments:')
print(rfm['RFM_Segment'].value_counts().head(10))

In [ ]:
# Customer Lifetime Value
print('\n' + '=' * 60)
print('CUSTOMER LIFETIME VALUE (CLV) ANALYSIS')
print('=' * 60)

clv = rfm[['Monetary', 'Frequency']].copy()
clv['Avg_Order_Value'] = clv['Monetary'] / clv['Frequency']
clv = clv.sort_values('Monetary', ascending=False)

print(f'\nCLV Statistics:')
print(clv.describe().round(2))

print(f'\nTop 10 Customers by CLV:')
print(clv.head(10).round(2))

In [ ]:
# Visualize RFM and CLV
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Recency distribution
axes[0, 0].hist(rfm['Recency'], bins=30, edgecolor='black', color='steelblue', alpha=0.7)
axes[0, 0].set_title('Recency Distribution', fontsize=11, fontweight='bold')
axes[0, 0].set_xlabel('Days Since Last Purchase')
axes[0, 0].set_ylabel('Number of Customers')

# Frequency distribution
axes[0, 1].hist(rfm['Frequency'], bins=20, edgecolor='black', color='lightcoral', alpha=0.7)
axes[0, 1].set_title('Frequency Distribution', fontsize=11, fontweight='bold')
axes[0, 1].set_xlabel('Number of Purchases')
axes[0, 1].set_ylabel('Number of Customers')

# Monetary distribution
axes[1, 0].hist(rfm['Monetary'], bins=30, edgecolor='black', color='lightgreen', alpha=0.7)
axes[1, 0].set_title('Monetary Distribution', fontsize=11, fontweight='bold')
axes[1, 0].set_xlabel('Total Spending ($)')
axes[1, 0].set_ylabel('Number of Customers')

# Frequency vs Monetary (Scatter)
scatter = axes[1, 1].scatter(rfm['Frequency'], rfm['Monetary'], alpha=0.6, c=rfm['Recency'], 
                             cmap='RdYlBu_r', s=100, edgecolor='black', linewidth=0.5)
axes[1, 1].set_title('Frequency vs Monetary Value', fontsize=11, fontweight='bold')
axes[1, 1].set_xlabel('Purchase Frequency')
axes[1, 1].set_ylabel('Total Spending ($)')
cbar = plt.colorbar(scatter, ax=axes[1, 1])
cbar.set_label('Recency (Days)', rotation=270, labelpad=20)

plt.tight_layout()
plt.show()

In [ ]:
# Hypothesis Testing: Are sales different across regions?
from scipy.stats import f_oneway, chi2_contingency

print('\n' + '=' * 60)
print('HYPOTHESIS TESTING')
print('=' * 60)

# ANOVA test: Sales across regions
north_sales = df_clean[df_clean['region'] == 'North']['sales']
south_sales = df_clean[df_clean['region'] == 'South']['sales']
east_sales = df_clean[df_clean['region'] == 'East']['sales']
west_sales = df_clean[df_clean['region'] == 'West']['sales']

f_stat, p_value = f_oneway(north_sales, south_sales, east_sales, west_sales)

print(f'\nANOVA Test: Sales across Regions')
print(f'  Null Hypothesis: Mean sales are equal across all regions')
print(f'  F-Statistic: {f_stat:.4f}')
print(f'  P-Value: {p_value:.6f}')
print(f'  Result: {"Reject null hypothesis" if p_value < 0.05 else "Fail to reject null hypothesis"}')

# Chi-square test: Category independence from Region
contingency_table = pd.crosstab(df_clean['category'], df_clean['region'])
chi2, p_val, dof, expected = chi2_contingency(contingency_table)

print(f'\nChi-Square Test: Category Independence from Region')
print(f'  Null Hypothesis: Category and Region are independent')
print(f'  Chi-Square Statistic: {chi2:.4f}')
print(f'  P-Value: {p_val:.6f}')
print(f'  Result: {"Reject null hypothesis" if p_val < 0.05 else "Fail to reject null hypothesis"}')

## 8. Business Insights and Recommendations

In [ ]:
print('=' * 70)
print('KEY BUSINESS INSIGHTS & RECOMMENDATIONS')
print('=' * 70)

# Calculate key metrics
total_revenue = df_clean['sales'].sum()
total_customers = df_clean['customer_id'].nunique()
avg_order_value = df_clean['sales'].mean()
total_orders = len(df_clean)
orders_per_customer = total_orders / total_customers

# Top performing categories
top_category = df_clean.groupby('category')['sales'].sum().idxmax()
top_category_sales = df_clean[df_clean['category'] == top_category]['sales'].sum()
top_category_pct = (top_category_sales / total_revenue) * 100

# Best performing region
top_region = df_clean.groupby('region')['sales'].sum().idxmax()
top_region_sales = df_clean[df_clean['region'] == top_region]['sales'].sum()
top_region_pct = (top_region_sales / total_revenue) * 100

# Customer segment insights
premium_pct = (df_clean[df_clean['customer_segment'] == 'Premium']['sales'].sum() / total_revenue) * 100

print(f'\n1. REVENUE & SALES METRICS')
print(f'   • Total Revenue: ${total_revenue:,.2f}')
print(f'   • Total Unique Customers: {total_customers:,}')
print(f'   • Total Orders: {total_orders:,}')
print(f'   • Average Order Value: ${avg_order_value:.2f}')
print(f'   • Orders per Customer: {orders_per_customer:.2f}')

print(f'\n2. PRODUCT PERFORMANCE')
print(f'   • Top Category: {top_category} ({top_category_pct:.1f}% of revenue)')
print(f'   • Top Category Revenue: ${top_category_sales:,.2f}')

print(f'\n3. GEOGRAPHIC INSIGHTS')
print(f'   • Best Performing Region: {top_region} ({top_region_pct:.1f}% of revenue)')
print(f'   • Regional Revenue: ${top_region_sales:,.2f}')

print(f'\n4. CUSTOMER SEGMENTATION')
print(f'   • Premium Customers Contribution: {premium_pct:.1f}% of revenue')
segment_breakdown = df_clean.groupby('customer_segment')['customer_id'].nunique()
for segment in segment_breakdown.index:
    pct = (segment_breakdown[segment] / total_customers) * 100
    print(f'     - {segment}: {segment_breakdown[segment]} customers ({pct:.1f}%)')

print(f'\n5. TREND ANALYSIS')
yearly_sales = df_clean.groupby('year')['sales'].sum()
growth_rate = ((yearly_sales.iloc[-1] - yearly_sales.iloc[-2]) / yearly_sales.iloc[-2]) * 100
print(f'   • Latest Year-over-Year Growth: {growth_rate:.1f}%')
print(f'   • Strongest Month: {df_clean[df_clean['month'] == df_clean.groupby('month')['sales'].sum().idxmax()]['month'].iloc[0]}')

In [ ]:
print('\n' + '=' * 70)
print('STRATEGIC RECOMMENDATIONS')
print('=' * 70)

recommendations = [
    {
        'title': '1. FOCUS ON HIGH-VALUE CUSTOMER SEGMENTS',
        'points': [
            f'   • Premium customers represent {premium_pct:.1f}% of revenue with only {segment_breakdown["Premium"] / total_customers * 100:.1f}% of customer base',
            '   • Implement VIP loyalty programs for premium customers',
            '   • Offer exclusive products and early access to new releases',
            '   • Consider personalized customer service for high-value accounts'
        ]
    },
    {
        'title': '2. OPTIMIZE PRODUCT CATEGORY STRATEGY',
        'points': [
            f'   • {top_category} dominates with {top_category_pct:.1f}% of revenue',
            '   • Invest in marketing campaigns for underperforming categories',
            '   • Bundle slow-moving categories with bestsellers',
            '   • Expand inventory for high-margin products in {top_category}'
        ]
    },
    {
        'title': '3. REGIONAL EXPANSION OPPORTUNITY',
        'points': [
            f'   • {top_region} region performs best ({top_region_pct:.1f}% of revenue)',
            '   • Identify growth potential in underperforming regions',
            '   • Tailor regional marketing strategies based on category preferences',
            '   • Consider regional shipping optimization for lower-performing areas'
        ]
    },
    {
        'title': '4. CUSTOMER RETENTION FOCUS',
        'points': [
            f'   • Average customer purchases only {orders_per_customer:.2f} times',
            '   • Implement email marketing and follow-up campaigns',
            '   • Create loyalty rewards program for repeat customers',
            '   • Target inactive customers with win-back campaigns'
        ]
    },
    {
        'title': '5. DATA-DRIVEN INVENTORY MANAGEMENT',
        'points': [
            '   • Use forecasting models to predict demand by category and region',
            '   • Implement dynamic pricing strategies based on demand patterns',
            '   • Monitor inventory turnover rates by product category',
            '   • Reduce stockouts for high-demand products during peak seasons'
        ]
    }
]

for rec in recommendations:
    print(f'\n{rec["title"]}')
    for point in rec['points']:
        print(point)

## 9. Summary & Export

In [ ]:
# Create comprehensive summary report
summary_stats = {
    'Metric': [
        'Total Revenue',
        'Total Orders',
        'Unique Customers',
        'Avg Order Value',
        'Orders per Customer',
        'Date Range',
        'Top Category',
        'Top Region'
    ],
    'Value': [
        f'${total_revenue:,.2f}',
        f'{total_orders:,}',
        f'{total_customers:,}',
        f'${avg_order_value:.2f}',
        f'{orders_per_customer:.2f}',
        f'{df_clean["order_date"].min().date()} to {df_clean["order_date"].max().date()}',
        top_category,
        top_region
    ]
}

summary_df = pd.DataFrame(summary_stats)

print('\n' + '=' * 70)
print('ANALYSIS SUMMARY')
print('=' * 70)
print(summary_df.to_string(index=False))

In [ ]:
# Export cleaned data
export_path = '../data/processed/cleaned_data.csv'
df_clean.to_csv(export_path, index=False)
print(f'✓ Cleaned dataset exported to: {export_path}')

# Export summary statistics
summary_df.to_csv('../data/processed/summary_statistics.csv', index=False)
print(f'✓ Summary statistics exported to: ../data/processed/summary_statistics.csv')

# Export RFM analysis
rfm.to_csv('../data/processed/rfm_analysis.csv')
print(f'✓ RFM analysis exported to: ../data/processed/rfm_analysis.csv')

print('\n' + '=' * 70)
print('ANALYSIS COMPLETE')
print('=' * 70)
print('\nAll visualizations, cleaned data, and analysis results have been saved.')
print('\nFiles generated:')
print('  • Cleaned Dataset: data/processed/cleaned_data.csv')
print('  • Summary Statistics: data/processed/summary_statistics.csv')
print('  • RFM Analysis: data/processed/rfm_analysis.csv')
print('\nNext Steps:')
print('  1. Review visualizations for presentations')
print('  2. Share insights with stakeholders')
print('  3. Implement recommendations for business optimization')
print('  4. Monitor KPIs monthly for performance tracking')